<a href="https://colab.research.google.com/github/AarthiThirumavalavan/DL/blob/main/CNN_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
!pip install keras-tuner

In [18]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [19]:
fashion_mnist = keras.datasets.fashion_mnist

In [20]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [21]:
train_images = train_images/255.0
test_images = test_images/255.0

In [22]:
train_images.shape

(60000, 28, 28)

In [23]:
train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [24]:
train_images.shape

(60000, 28, 28, 1)

In [25]:
test_images.shape

(10000, 28, 28, 1)

In [26]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [27]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters


In [28]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [29]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.878000020980835

Best val_accuracy So Far: 0.9098333120346069
Total elapsed time: 00h 02m 34s
INFO:tensorflow:Oracle triggered exit


In [30]:
model = tuner_search.get_best_models(num_models=1)[0]

In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        13856     
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 32)                589856    
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 604,522
Trainable params: 604,522
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1804 - accuracy: 0.9330 - val_loss: 0.2479 - val_accuracy: 0.9160
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1442 - accuracy: 0.9470 - val_loss: 0.2773 - val_accuracy: 0.9065
Epoch 6/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1135 - accuracy: 0.9575 - val_loss: 0.2700 - val_accuracy: 0.9130
Epoch 7/10
1688/1688 [==============================] - 6s 3ms/step - loss: 0.0918 - accuracy: 0.9663 - val_loss: 0.3179 - val_accuracy: 0.9133
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0723 - accuracy: 0.9734 - val_loss: 0.3376 - val_accuracy: 0.9117
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0553 - accuracy: 0.9799 - val_loss: 0.3728 - val_accuracy: 0.9090
Epoch 10/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0457 - accuracy: 0.9831 - val_loss: 0.3956 - val_accuracy